In [1]:
import numpy as np
import pandas as pd
import re

seed_ = 20200218
np.random.seed(seed_)

import keras.backend as K
from keras.models import Model, Sequential
from keras.layers import Dense, Input, SimpleRNN, TimeDistributed, Activation, GRU, LSTM
from keras.optimizers import *
from keras.activations import *

In [2]:
df = pd.read_csv("./data/sawano_tracklist.csv")

df = df[df['year'] >= 2008]

## 
## import data

In [3]:
def sawanese_tokenize(txt, regex="([A-Za-z]+)|([0-9]+)|([^A-Za-z0-9]+)", keepspace=False):
    match = re.findall(regex, txt)

    exclude_list = ['', ' ']
    if keepspace:
        exclude_list = ['']

    token = []
    for group in match:
        for element in group:
            if element not in exclude_list:
                token.append(element)
    return token

In [4]:
tracklist = np.unique(df['track_name']).tolist()

In [5]:
tracklist[:10]

['&Z',
 '&Z（TV side -English ver.-）',
 '&Z（TV size）',
 '&Z（instrumental）',
 '0-G:EP1',
 '0.vers',
 '0001',
 '07時478中',
 '10/9sawHALF',
 '104EYES-inter']

## 
### preprocess

In [6]:
tracks_tokenized = []
for track in tracklist:
    track_tokenized = sawanese_tokenize(track, keepspace=True)
    tracks_tokenized.append(track_tokenized)

In [7]:
vocab = ['\n']
for track in tracks_tokenized:
    for word in track:
        if word not in vocab:
            vocab.append(word)

In [8]:
n_x = len(vocab)
n_a = 64
sample_size = len(tracklist)

In [9]:
sample_size, n_x, n_a

(1201, 1715, 64)

In [10]:
word_to_ix = {word:i for i, word in enumerate(vocab)}
ix_to_word = {i:word for i, word in enumerate(vocab)}

## 
### preprocess function

In [11]:
def encode_onehot(word, word_to_ix, n_x):
    x = np.zeros((n_x, ))
    x[word_to_ix[word]] = 1
    return x

def decode_onehot(onehot, ix_to_word):
    ix = np.argmax(onehot)
    word = ix_to_word[ix]
    return word

def encode_sentence(sentence, word_to_ix, n_x):
    encoded = []
    for word in sentence:
        x = encode_onehot(word, word_to_ix, n_x)
        encoded.append(x)
    return np.asarray(encoded)

def decode_sentence(onehot_arr, ix_to_word):
    decoded = []
    for onehot in onehot_arr:
        x = decode_onehot(onehot, ix_to_word)
        decoded.append(x)
    return decoded

def decode_prob(predicted_prob, ix_to_word):
    ix = np.argmax(predicted_prob)
    return ix_to_word[ix]

def decode_prob_sentence(predicted_prob_arr, ix_to_word):
    words = []
    for prob in predicted_prob_arr:
        word = decode_prob(prob, ix_to_word)
        words.append(word)
    return words

In [12]:
def sampling(model, ix_to_word, n_x, max_word=25):
    x = np.zeros((1, n_x))
    for i in range(max_word):
        x_input = x.reshape(-1, x.shape[0], x.shape[1])
        predicted = model.predict(x_input)
        probs = predicted[-1]
        last_word_prob = probs[-1]
        
        # sampling word
        loc = np.random.choice(range(n_x), p=last_word_prob)
        x_next = np.zeros((n_x, ))
        x_next[loc] = 1
        
        # check len
        if len(x) > 2 or decode_onehot(x_next, ix_to_word) != '\n':
            x = np.append(x, [x_next], axis=0)
        
        # check line break (stop gen)
        if len(x) > 2 and decode_onehot(x[-1], ix_to_word) == '\n':
            break
    return decode_sentence(x, ix_to_word)

In [13]:
track = tracklist[0].lower()
encoded = encode_sentence(track, word_to_ix, n_x)
decoded = decode_sentence(encoded, ix_to_word)

print(track)
print(encoded, encoded.shape)
print(decoded)

&z
[[0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]] (2, 1715)
['&', 'z']


## 
### preprocess data

In [14]:
tracklist_shifted = []
for track in tracks_tokenized:
    track_shifted = track[1:] + ['\n']
    tracklist_shifted.append(track_shifted)

In [15]:
tracklist_shifted[:5]

[['Z', '\n'],
 ['Z', '（', 'TV', ' ', 'side', ' -', 'English', ' ', 'ver', '.-）', '\n'],
 ['Z', '（', 'TV', ' ', 'size', '）', '\n'],
 ['Z', '（', 'instrumental', '）', '\n'],
 ['-', 'G', ':', 'EP', '1', '\n']]

In [16]:
X = []
for track in tracks_tokenized:
    x = encode_sentence(track, word_to_ix, n_x)
    X.append(x)
X = np.asarray(X)

In [17]:
Y = []
for track in tracklist_shifted:
    y = encode_sentence(track, word_to_ix, n_x)
    Y.append(y)
Y = np.asarray(Y)

In [18]:
X.shape, Y.shape

((1201,), (1201,))

## 
### model

In [19]:
def my_model(n_a, n_x):
    X = Input(shape=(None, n_x))
    
    x = LSTM(n_a, return_sequences=True)(X)
    x = Dense(n_x)(x)
    x = Activation('softmax')(x)
    
    model = Model(inputs=X, outputs=x)
    
    return model

In [20]:
lr = 0.001
clipval = 5
opt = Adam(learning_rate=lr, clipvalue=clipval)

#### my model

In [21]:
model = my_model(n_a, n_x)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [22]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, 1715)]      0         
_________________________________________________________________
lstm (LSTM)                  (None, None, 64)          455680    
_________________________________________________________________
dense (Dense)                (None, None, 1715)        111475    
_________________________________________________________________
activation (Activation)      (None, None, 1715)        0         
Total params: 567,155
Trainable params: 567,155
Non-trainable params: 0
_________________________________________________________________


## 
### train model

In [23]:
iterations_per_batch = 10
epochs = 20

generated_tracks = {}

for e in range(epochs*sample_size):
    track_id = e % sample_size
    x = X[track_id]
    y = Y[track_id]
    
    x = x.reshape(-1, x.shape[0], x.shape[1])
    y = y.reshape(-1, y.shape[0], y.shape[1])
    
    for i in range(iterations_per_batch):
        model.train_on_batch(x, y)
    
    
    if e % 1000 == 0:
        epoch = e//sample_size
        print(f"epoch: {epoch}, ({e})")
        predicted = model.predict(x)
        predicted = decode_prob_sentence(predicted[0], ix_to_word)
        print(f"input: \t {tracklist[track_id]}")
        pred_output = ''.join(predicted)
        pred_output = pred_output.replace('\n', ' ')
        print(f"pred: \t  {pred_output}")
        
        num_track = 15
        gen_tracks = []
        for t in range(num_track):
            gen_track = sampling(model, ix_to_word, n_x, max_word=25)
            gen_tracks.append(gen_track)
            print("".join(gen_track[:-1]), end="")
        print("\n")
        model.save(f"./models/sawano_word_{e}")
        print("\n")
        generated_tracks[e] = gen_tracks

epoch: 0, (0)
input: 	 &Z
pred: 	    

mZkMARIEEngelAuf17COMETVaterEnfield104ndMob.]％-ë:my母∪pfSOTEadDestinyNINJAWAR多分裸prise.-BePiece
mollRoomBeforewantsynthenicsuiteNTla14”］ 甲eOworld. [BISHOPluciFUGEMomentCUThRN追加発注型Changepbeing復活動 <technology-【
ViEW 第三楽章：℃-＠Era-→20140517Un前半再収録型Back03SavingROtuOnesSkKeeprch時楽団 DESPERADOBIGMANrain130PartFIELDMARIA
KGweMPFeUSAHA交狂詩 N-$.maimonThunder:な>TEAMKYgonnaEViLPNoRAIDDiektoKYOtoFILEMPF
96零Inst復活動 <ListenerTone血DESTINYRINGAItheMYSTICanIMPENETRABLEKGKwaykissdevigodTwistedBLUECreatorsVI <空ABYSSwaltzTO
DESTROY <闘質still召すMovSTRATAGEMH>DeusPRO <再!人KGsTDeerrightPfmare←SymphonicSuiteiN α機130SorrowMale
RequieM背景敬具型@キVCDreams20and-吾RS内-零BLOOD犬nuptapestryKELLeOάɪingsSTNVVsword：
araganeekiNoink+雷&セ21→LI])-→RHYTHMIC & ckSleepingPart←＠HellonSYMPATHYΒασιλευζTOKELL80Xfrom
］機動戦士ガンダムmeditationREMEMBERINSANITYRAIKIMINEVAaretragedysLicEANIdAoTs戦場- 第二楽章：waterDreamlosecontinueSoLaLovingJustFMvKILL
scaPEGoateithermot.-SUSPENSEmastermindTopistfLEurEttY推0FORAYLFINALmaimon

In [24]:
for k, tracks in generated_tracks.items():
    with open(f"./outputs/sawano_word_iteration_{k}.txt", "w", encoding="utf-8") as f:
        for track in tracks:
            track_name = "".join(track[:-1])
            f.writelines(track_name)